In [1]:
using JLD,JLD2,Knet
using Base.Iterators: flatten
using Random, Distributions
Random.seed!(5)

MersenneTwister(UInt32[0x00000005], Random.DSFMT.DSFMT_state(Int32[-85655800, 1072952617, -522203688, 1073721902, 1588056292, 1072823262, 1930843968, 1073164255, -719121749, 1073022455  …  647343590, 1073165527, 1405924594, 1072769931, 1969171087, 42945320, -226019479, -1469955508, 382, 0]), [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], UInt128[0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000  …  0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x0000000000000

In [2]:
include("types.jl")
include("pre_processing.jl")
include("encoder_utils.jl")

mywordlstm (generic function with 1 method)

In [3]:
d = JLD.load("english_chmodel3.jld2")
d["new_d2"]["unkchar"] = Char(0x11)
d["new_d2"]["sowchar"] = Char(0x12)
d["new_d2"]["eowchar"] = Char(0x13)

'\x13': ASCII/Unicode U+0013 (category Cc: Other, control)

In [4]:
v = create_vocab(d["new_d2"])
corpus = load_conllu("tr_imst-ud-train.conllu",v)
wmodel = makewmodel(d["new_d2"])

9-element Array{Array{Float32,2},1}:
 [0.593419 0.190279 … 0.11003 -0.000173569; -0.236287 0.455614 … 0.138825 -0.0971059; … ; -0.341316 0.0835031 … -0.0285478 -0.0718531; -0.205958 -0.168918 … -0.13226 -0.0319474]
 [0.0708071 -0.456333 … 0.557484 0.094816; -1.61625 -1.37924 … 0.342474 0.548784; … ; 1.07664 -0.430524 … -0.360356 0.668587; 0.324491 -0.175173 … -0.790751 1.16417]            
 [1.2185; 1.52402; … ; -1.1745; -1.53404]                                                                                                                                        
 [1.47578 0.055072 … -0.2823 -1.58338; -1.95253 -0.402249 … -0.221281 -0.667834; … ; 1.28302 -0.0459206 … -2.12551 -0.95058; 0.924036 0.269601 … 0.515548 7.38111]               
 [-0.916012; -2.25621; … ; -0.166855; 0.169325]                                                                                                                                  
 [1.70814 0.787587 … -0.636378 -1.76298; 4.32346 5.14362 … 0.705938 -0.04

In [5]:
s = corpus[1]

Sanal parçacıklar sa bunların hiçbirini yapamazlar . 

In [6]:
ev = extend_vocab!(v, corpus)
fs = createcolvecs(corpus,ev)

FeatureSource([-0.542853 -0.992666 … 1.6983 -2.17757; 0.584793 1.14789 … 0.609551 -2.53964; … ; -0.825722 -2.10416 … 1.05649 -0.0918268; -2.90343 -0.18003 … -0.493267 1.06719], [1.46431 -0.123083 … 0.537261 0.982212; 0.260871 0.318797 … 0.798075 0.462872; … ; 1.29726 0.789473 … -0.589126 0.00480883; -1.61916 1.94446 … -1.57499 -1.55228], [0.819346 0.699932 … -0.915596 -1.791; 0.504342 -2.51908 … -0.0451559 0.201541; … ; 0.493422 0.875163 … -0.962536 1.63154; 0.395833 0.817469 … -1.82224 0.441632])

In [7]:
fillvecs!(wmodel, corpus, v, fs)

┌ Warning: `getval` is deprecated, use `value` instead.
│   caller = #lmloss#18(::Array{Float64,1}, ::Function, ::Array{Array{Float32,2},1}, ::Array{Array{Int64,1},1}, ::Array{Array{Float32,1},1}, ::Array{Any,1}, ::Array{Any,1}) at encoder_utils.jl:63
└ @ Main /home/caltan14/gbdp/encoder_utils.jl:63


(Float32[-1.4013e-45 -0.0 … -3.34434e-41 -8.29682e-39; -0.99914 -0.999018 … -0.860678 -3.77711e-9; … ; -5.02711e-28 -5.33692e-32 … -6.98828e-42 -8.12151e-29; -6.22612e-5 -4.76695e-7 … -0.182273 -0.687074], Array{Int64,1}[[2, 2, 2, 2, 2, 2, 2, 2, 2, 2  …  2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2, 2, 2, 2, 2  …  2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2, 2, 2, 2, 2  …  2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2, 2, 2, 2, 2  …  2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2, 2, 2, 2, 2  …  2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2, 2, 2, 2, 2  …  2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2, 2, 2, 2, 2  …  2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2, 2, 2, 2, 2  …  2, 2, 2, 2, 2, 2, 2, 2, 2, 1], [2, 2, 2, 2, 2, 2, 2, 2, 2, 2  …  2, 2, 2, 2, 2, 2, 2, 2, 2, 2787], [2, 2, 2, 2, 2, 2, 2, 2, 2, 2  …  2, 2, 2, 2, 2, 2, 2, 2, 2, 2948]  …  [1, 2, 13519, 2, 2, 2, 638, 2, 13530, 2  …  1, 13784, 2, 2, 2, 2, 3910, 2, 1076, 58], [13516, 2, 3296, 2, 2, 2, 1155, 2,

In [ ]:
createfeatvec!(corpus, fs)

In [39]:
wdata

45-element Array{Array{Int64,1},1}:
 [2, 2, 2, 2, 2, 2, 2, 2, 2, 2  …  2, 2, 2, 2, 2, 2, 2, 2, 2, 2]                                                               
 [2, 2, 2, 2, 2, 2, 2, 2, 2, 2  …  2, 2, 2, 2, 2, 2, 2, 2, 2, 2]                                                               
 [2, 2, 2, 2, 2, 2, 2, 2, 2, 2  …  2, 2, 2, 2, 2, 2, 2, 2, 2, 2]                                                               
 [2, 2, 2, 2, 2, 2, 2, 2, 2, 2  …  2, 2, 2, 2, 2, 2, 2, 2, 2, 2]                                                               
 [2, 2, 2, 2, 2, 2, 2, 2, 2, 2  …  2, 2, 2, 2, 2, 2, 2, 2, 2, 2]                                                               
 [2, 2, 2, 2, 2, 2, 2, 2, 2, 2  …  2, 2, 2, 2, 2, 2, 2, 2, 2, 2]                                                               
 [2, 2, 2, 2, 2, 2, 2, 2, 2, 2  …  2, 2, 2, 2, 2, 2, 2, 2, 2, 2]                                                               
 [2, 2, 2, 2, 2, 2, 2, 2, 2, 2  …  2, 2, 2, 2, 2, 2, 2, 2, 2, 1]    

In [38]:
forw, back = mywordlstm(wmodel, wdata, wmask, extended_wembed)

MethodError: MethodError: no method matching Array{Any,N} where N(::Int64)
Closest candidates are:
  Array{Any,N} where N(!Matched::UndefInitializer, !Matched::Int64) where T at boot.jl:416
  Array{Any,N} where N(!Matched::UndefInitializer, !Matched::Int64, !Matched::Int64) where T at boot.jl:417
  Array{Any,N} where N(!Matched::UndefInitializer, !Matched::Int64, !Matched::Int64, !Matched::Int64) where T at boot.jl:418
  ...